## Data Manipulation v2.0

In the first version of this file, I attempted to combine 3 datasets taken from the CDC website showing U.S. cause of death data for 1968-1978, 1979-1998, and 1999-2014. In that attempt, I ended up giving up on the 68-78 dataset because it was hard to find the causes of death that correspond to the same causes in the later datasets due to differences in naming. I did a good bit of data wrangling to match up the 79-98 and 99-14 datasets, and I managed to combine things to get a common Top 10 list of causes to use. However, when visualizing the data, there were some glaring jumps in the data where the two datasets met:

<center><img src="bad_data.png"><br><img src="bad_atherosclerosis_data.png"></center>

Therefore, in this version, I am only going to use the most recent dataset, which is at least consistent with itself.

In [14]:
import numpy as np
import pandas as pd

def load_df(start_year, end_year):
    df = pd.DataFrame.from_csv(
        'Compressed Mortality, {}-{}.txt'.format(str(start_year), str(end_year)),
        sep='\t',
        header=0,
        index_col=False
    )
    
    # clean up
    del df['Year Code']
    del df['Cause of death Code']
    del df['Notes']
    df = df[pd.notnull(df['Year'])]
    df.Year = df.Year.astype(int)
    df.Deaths = df.Deaths.astype(int)
    df.Population = df.Population.astype(int)
    df['Crude Rate'] = df['Crude Rate'].astype(str)
    df['Crude Rate Unreliable'] = df['Crude Rate'].str.contains('Unreliable')
    df['Crude Rate'] = df['Crude Rate'].str.split(' ').str.get(0).astype(float)
    
    return df

df = load_df(1999,2014)

df.head()

,Year,Cause of death,Deaths,Population,Crude Rate,Crude Rate Unreliable
0,1999,"Cholera, unspecified",1,279040168,0.0,True
1,1999,Salmonella enteritis,13,279040168,0.0,True
2,1999,Salmonella septicaemia,23,279040168,0.0,False
3,1999,Localized salmonella infections,1,279040168,0.0,True
4,1999,"Salmonella infection, unspecified",1,279040168,0.0,True


In [15]:
deaths_by_year = df.groupby(['Year','Cause of death']) \
                   .aggregate(sum) \
                   .sort_values(by='Deaths', ascending=False) \
                   .loc[:,['Deaths']]
deaths_by_year.loc[2000].head(10)

,Deaths
Cause of death,
Atherosclerotic heart disease,222368.0
"Acute myocardial infarction, unspecified",192539.0
"Bronchus or lung, unspecified - Malignant neoplasms",154157.0
"Chronic obstructive pulmonary disease, unspecified",94394.0
"Stroke, not specified as haemorrhage or infarction",90379.0
"Atherosclerotic cardiovascular disease, so described",69554.0
"Pneumonia, unspecified",54877.0
Congestive heart failure,51546.0
"Alzheimer's disease, unspecified",47614.0


Because they are all very similar, I am going to group together "Atherosclerotic heart disease", "Acute myocardial infarction, unspecified" (heart attack), "Atherosclerotic cardiovascular disease, so described", and "Congestive heart failure" into one group: "Heart disease"

In [16]:
for cause in df['Cause of death'].unique():
    if (
        cause == "Atherosclerotic heart disease" or
        cause == "Acute myocardial infarction, unspecified" or
        cause == "Atherosclerotic cardiovascular disease, so described" or
        cause == "Congestive heart failure"
    ):
        df['Cause of death'][df['Cause of death'] == cause] = 'Heart disease'

deaths_by_year = df.groupby(['Year','Cause of death']) \
                   .aggregate(sum) \
                   .sort_values(by='Deaths', ascending=False) \
                   .loc[:,['Deaths']]
deaths_by_year.loc[2000].head(10)

,Deaths
Cause of death,
Heart disease,536007.0
"Bronchus or lung, unspecified - Malignant neoplasms",154157.0
"Chronic obstructive pulmonary disease, unspecified",94394.0
"Stroke, not specified as haemorrhage or infarction",90379.0
"Pneumonia, unspecified",54877.0
"Alzheimer's disease, unspecified",47614.0
"Colon, unspecified - Malignant neoplasms",46652.0
"Breast, unspecified - Malignant neoplasms",42290.0
"Unspecified diabetes mellitus, without complications",40424.0


Likewise, "Bronchus or lung, unspecified - Malignant neoplasms", "Chronic obstructive pulmonary disease, unspecified", and "Pneumonia, unspecified" are all examples of lung disease.

In [17]:
for cause in df['Cause of death'].unique():
    if (
        cause == "Bronchus or lung, unspecified - Malignant neoplasms" or
        cause == "Chronic obstructive pulmonary disease, unspecified" or
        cause == "Pneumonia, unspecified"
    ):
        df['Cause of death'][df['Cause of death'] == cause] = 'Lung disease'

deaths_by_year = df.groupby(['Year','Cause of death']) \
                   .aggregate(sum) \
                   .sort_values(by='Deaths', ascending=False) \
                   .loc[:,['Deaths']]
deaths_by_year.loc[2000].head(10)

,Deaths
Cause of death,
Heart disease,536007.0
Lung disease,303428.0
"Stroke, not specified as haemorrhage or infarction",90379.0
"Alzheimer's disease, unspecified",47614.0
"Colon, unspecified - Malignant neoplasms",46652.0
"Breast, unspecified - Malignant neoplasms",42290.0
"Unspecified diabetes mellitus, without complications",40424.0
Unspecified dementia,32540.0
Malignant neoplasm of prostate,31078.0


Let's also combine Alzheimer's and dementia.

In [18]:
for cause in df['Cause of death'].unique():
    if (
        cause == "Alzheimer's disease, unspecified" or
        cause == "Unspecified dementia"
    ):
        df['Cause of death'][df['Cause of death'] == cause] = "Alzheimer's/dementia"

deaths_by_year = df.groupby(['Year','Cause of death']) \
                   .aggregate(sum) \
                   .sort_values(by='Deaths', ascending=False) \
                   .loc[:,['Deaths']]
deaths_by_year.loc[2000].head(10)

,Deaths
Cause of death,
Heart disease,536007.0
Lung disease,303428.0
"Stroke, not specified as haemorrhage or infarction",90379.0
Alzheimer's/dementia,80154.0
"Colon, unspecified - Malignant neoplasms",46652.0
"Breast, unspecified - Malignant neoplasms",42290.0
"Unspecified diabetes mellitus, without complications",40424.0
Malignant neoplasm of prostate,31078.0
"Pancreas, unspecified - Malignant neoplasms",28980.0


And finally, for the rest of the top 10, let's rename them to something more palatable.

In [19]:
for cause in df['Cause of death'].unique():
    if cause == "Colon, unspecified - Malignant neoplasms":
        df['Cause of death'][df['Cause of death'] == cause] = "Colon cancer"
    elif cause == "Breast, unspecified - Malignant neoplasms":
        df['Cause of death'][df['Cause of death'] == cause] = "Breast cancer"
    elif cause == "Unspecified diabetes mellitus, without complications":
        df['Cause of death'][df['Cause of death'] == cause] = "Diabetes"
    elif cause == "Malignant neoplasm of prostate":
        df['Cause of death'][df['Cause of death'] == cause] = "Prostate cancer"
    elif cause == "Pancreas, unspecified - Malignant neoplasms":
        df['Cause of death'][df['Cause of death'] == cause] = "Pancreatic cancer"
    elif cause == "Septicaemia, unspecified":
        df['Cause of death'][df['Cause of death'] == cause] = "Septicaemia"
    elif cause == "Stroke, not specified as haemorrhage or infarction":
        df['Cause of death'][df['Cause of death'] == cause] = "Stroke"

deaths_by_year = df.groupby(['Year','Cause of death']) \
                   .aggregate(sum) \
                   .sort_values(by='Deaths', ascending=False) \
                   .loc[:,['Deaths']]
deaths_by_year.loc[2000].head(10)

,Deaths
Cause of death,
Heart disease,536007.0
Lung disease,303428.0
Stroke,90379.0
Alzheimer's/dementia,80154.0
Colon cancer,46652.0
Breast cancer,42290.0
Diabetes,40424.0
Prostate cancer,31078.0
Pancreatic cancer,28980.0


In [20]:
# make a csv where each row is a disease and each column is a year
diseases = np.unique(deaths_by_year.loc[2014].head(10).index)
start_year = 1999
end_year = 2014
header = ['disease'] + [str(year) for year in range(start_year,end_year+1)]
csv_list = [header]
for disease in diseases:
    row = [disease] + [deaths_by_year.loc[(year, disease)].values[0] for year in range(start_year,end_year+1)]
    csv_list.append(row)

import csv
with open('death_data_year_cols.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(csv_list)

In [21]:
# make a csv where each row is a year and each column is a disease
header = ['year'] + [disease for disease in diseases]
csv_list = [header]
for year in range(start_year,end_year+1):
    row = [year] + [deaths_by_year.loc[(year, disease)].values[0] for disease in diseases]
    csv_list.append(row)

with open('death_data_cause_cols.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(csv_list)